In [ ]:
import requests

## Suche

Der einfachste Weg, den gewünschten API-Aufruf zu konstruieren, führt über die Web-Oberfläche. Hier können die zunächst Suchparameter eingestellt werden, dann per Rechtsklick mit gedrückter Strg-Taste das Kontextmenü aufrufen und unter *Export* das gewünschte Format wählen. Ein Klick öffnet das Suchergebnis im gewünschten Format.

Die Webfrontend-Ansicht, die unter der URL <https://berlin.museum-digital.de/objects?s=place:10651> aufrufbar ist, entspricht <https://berlin.museum-digital.de/json/objects?&s=place%3A10651>.

In [ ]:
requests.get("https://berlin.museum-digital.de/json/objects?s=place:10651").json()

## Mehr als 24 Objekte auf einen Streich

Standardmäßig gibt das Webinterface und die API 24 Objekte je Aufruf aus. Dieser Wert kann auf max. 100 erhöht werden und mit dem Parameter `gbreitenat` übergeben werden. Über die Treffer kann mit dem Parameter `startwert` iteriert werden, indem `startwert` bei jedem Aufruf um `gbreitenat` erhöht wird. Um sich also eine größere Menge (sprich i.d.R. mehr als 100) an Objekten ausgeben zu lassen, bietet sich folgendes Vorgehen an: https://nat.museum-digital.de/objects?instnr=22&section=results_list&mode=grid&gbreitenat=100

In [ ]:
apiURL = "https://nat.museum-digital.de/json/objects?&s=gabel"

In [ ]:
objektIDs = list()

rows = 100
offset = 0
while True:   
    res = requests.get(apiURL + f"&startwert={offset}&gbreitenat={rows}")
    offset += rows
    if 'status' in res.json() and res.json()['status'] == 'Error':
        break
    else:
        for _ in res.json():
            objektIDs.append(_['objekt_id'])

In [ ]:
objektIDs = list()

rows = 100
offset = 0
while True:   
    res = requests.get(apiURL + f"&startwert={offset}&gbreitenat={rows}")
    offset += rows
    if 'status' in res.json() and res.json()['status'] == 'Error':
        break
    else:
        for _ in res.json():
            objektIDs.append(_['objekt_id'])

Mithilfe der gesammelten IDs kann nun ein Gesamtabzug der Daten erstellt werden.

In [ ]:
from tqdm import tqdm # Zeigt den Forschritt an
dump = []
for ID in tqdm(objektIDs):
    obj = requests.get(f"https://nat.museum-digital.de/json/object/{ID}").json()
    dump.append(obj)

In [ ]:
herstellungsdaten = []
for obj in dump:
    for event in obj['object_events']:
        if event['event_type'] == 1:
            if 'time' in event:
                time_end =  event['time']['time_end']
                try:
                    herstellungsdaten.append(int(time_end))
                except Exception as e:
                    pass

In [ ]:
from matplotlib import pyplot as plt

plt.hist(herstellungsdaten, bins = 50)
plt.show()

## Liste der Institutionen

In [ ]:
res = requests.get("https://nat.museum-digital.de/institutions?output=json")

In [ ]:
# Die Institutionen mit den 5 größten Online-Beständen

# nach größe Sortieren
instSortiert = sorted([_ for _ in res.json()], key = lambda x:x['institution_objects'], reverse = True)

for _ in instSortiert[:6]:
    print(f"{_['institution_id']}, {_['institution_name']} ({_['institution_objects']} Objekte)")

### Sammlungen

In [ ]:
res = requests.get("https://global.museum-digital.org/json/collections")

In [ ]:
collections = []
for inst in res.json():
    for coll in inst['collection']:
        collections.append(coll['collection_id'])
        if coll['collection_no_of_subcollections'] > 0:
            for subcoll in coll['collection_subcollections']:
                collections.append(subcoll['subcollection_id'])

In [ ]:
len(collections)

## Public-Domain-Checker

Objekte, deren Urheber:innen seit 70 Jahren tot sind, sind gemeinfrei.
Nach § 68 dürften auch im Falle von Reproduktionsfotografien keine Rechte entstehen, somit sind auch die Digitalisate gemeinfrei.
Im Folgenden, sollen Objekte von Museum-Digital mit einer eher unterkomplexen Heuristik auf Gemeinfreiheit überprüft werden. Es soll lediglich das späteste urheberrechtsrelevante Datum ermittelt werden.
Liegt dieses mehr als 120 Jahre zurück, wird das Objekt als potentiell gemeinfrei angesehen.
Falls eine anderslautende Lizenz für die Abbildung vergeben wurde, wird eine Warnung ausgegeben.

In [ ]:
def pdChecker(objID):
    from datetime import datetime
    currentYear = datetime.now().year

    # Urheberrelevante Event-Types (https://event-types.museum-digital.org/)
    event_types = [1,3,4,7,9,10,12,19,35,39,48]
    
    # freie Lizenzen
    freeLicenses = ["CC0", ]

    res = requests.get(f"https://berlin.museum-digital.de/json/object/{objID}")

    times = []

    for event in res.json()['object_events']:   

        if event['event_type'] in event_types:
            
            # Eventdaten
            if 'time' in event:
                try:
                    times.append(int(event['time']['time_end']))
                except:
                    pass
            
    lizenz = [o['rights'] for o in res.json()['object_images']]
    print(lizenz)

    if len(times) > 0:
        if currentYear - max(times) < 120:
            print("geminfrei")
        else:
            print("nicht gemeinfrei")
    else:
        print("keine Daten vorhanden")
    
        print(objID, " vor mehr als 120 Jahren hergestellt, also wahrscheinlich gemeinfrei.")

In [ ]:
pdChecker(40495)

In [ ]:
def diedIn(persID):
    res = requests.get(f"https://term.museum-digital.de/md-de/persinst/{persID}/json")
    try:
        return int(res.json().get('year_of_death'))
    except:
        return currentYear

In [ ]:
def pdChecker(objID):
    from datetime import datetime

    currentYear = datetime.now().year

    res = requests.get(f"https://berlin.museum-digital.de/json/object/{objID}")

    times = []

    # Urheberrelevante Event-Types (https://event-types.museum-digital.org/)
    event_types = [1,3,4,7,9,10,12,19,35,39,48]

    for event in res.json()['object_events']:   

        if event['event_type'] in event_types:
            # Eventdaten
            if 'time' in event:
                try:
                    times.append({
                        'year' : int(event['time']['time_end']),
                        'actorName' : event['people']['people_name'],
                        'actorID' : event['people']['people_id'],
                        'eventType' : event['event_type']
                    })

                except:
                    pass


    if (currentYear - max(times, key=lambda x:x['year'])['year']) > 120:
        print(objID, " vor mehr als 120 Jahren hergestellt, also wahrscheinlich gemeinfrei.")
    else:
        actorDeathYears = []
        for t in times:
            died = diedIn(t['actorID'])
            if died:
                actorDeathYears.append(died)
        if all(currentYear - YoD > 70 for YoD in actorDeathYears):
            print(objID, " alle Urheber:innen mehr als 70 Jahre tot, also wahrscheinlich gmeeinfrei.")
        else:
            print(objID, " vermutlich noch nicht gemeinfrei.")

In [ ]:
datenabzug = []
collID = 64
offset = 0
rows = 100


while True:
    res = requests.get(f"https://berlin.museum-digital.de/json/objects?&s=collection%3A{collID}&&startwert={offset}&gbreitenat={rows}")
    if 'status' in res.json():
        break
    for ID in res.json():
        datenabzug.append(ID.get('objekt_id'))
    offset += rows
    
for obj in datenabzug:
    pdChecker(obj)
    input()

## Liste aller Inventarnummern einer Instanz erzeugen

Im Folgenden soll eine Liste aller Inventarnummern auf der Berliner Museum-Digital-Instanz generiert werden.

In [ ]:
apiURL = "https://berlin.museum-digital.de/json/objects?&s="

In [ ]:
from collections import Counter, defaultdict

In [ ]:
inventarnummern_je_institution = defaultdict(list)

rows = 100
offset = 0
while True:   
    res = requests.get(apiURL + f"&startwert={offset}&gbreitenat={rows}")
    offset += rows
    if 'status' in res.json() and res.json()['status'] == 'Error':
        break
    else:
        for _ in res.json():
            inventarnummern_je_institution[_['institution_id']].append(_['objekt_inventarnr'])
    print(len(inventarnummern_je_institution))

In [ ]:


c = Counter(inventarnummern)
dubletten = [(k,v) for k,v in c.items() if v > 1]

In [ ]:
dubletten

In [ ]:
for k,v in c.items():
    if v > 1:
        print(k,v)